In [1]:
import sys
import datetime

In [2]:
%load_ext autoreload

In [3]:
%autoreload 1

In [4]:
%aimport main

In [5]:
%aimport

Modules to reload:
main

Modules to skip:



# Outline for Djikstra:
1. Mark all nodes as unvisited
2. Set the distance of the starting node as 0, and infinity for all other nodes
3. Set the starting node as the current node
4. Visit each of the neighbors of the current node
    a. For each neighbor, calculate the distance to that node traveling through the current node
    b. If this distance is less then the current distance recorded for that node, update the record accordingly
5. . Mark the current node as "visited"
6. Of the unvisited nodes, set the one with the smallest distance to the current node
7. Repeat steps 4 through 6 until one of the following:
    a. The algorithm terminates when the destination node is the current node
    b. Alternatively, if the smallest distance of the unvisited nodes is, then no path exists to the destination node.

In [6]:
mm = main.main()
mm.read("ex.txt")
mm.pathlength()

Journey Completed! Process took: 6


In [7]:
mm = main.main()
mm.read("input.txt")
mm.pathlength()

Journey Completed! Process took: 20569


In [48]:
mm = main.main()
mm.read("input.txt")
print(len(mm.nodes))
paths2zzzs = [(key, values) for key, values in list(mm.neighbors.items()) if ((values[0][-1]=="Z") or (values[1][-1]=="Z"))]
print(len(paths2zzzs))
mm.pathlength2()

790
6
 On node  XVL   789  of 790     Node mappings completed!!!!


In [52]:
zpaths = [(key, zzzs) for key, zzzs in list(mm.p2node_zzzs.items()) if zzzs != []]
print(len(zpaths))

12


In [53]:
from collections import Counter

In [54]:
Counter(mm.p2node_zcount_maps.values())

Counter({0: 778, 1: 12})

In [55]:
#Make a better mapping
#Starting nodes
#len(directions)
starting_nodes = [node for node in mm.nodes if node[-1]=="A"]
print(len(starting_nodes))

6


## V2 - Increasing Speed!

In [92]:
cur_nodes = starting_nodes
cycles = 0

start = datetime.datetime.now()
ncycles = 10**6 #search size; number of iterations until termination

ncycles = 2 * 10**9
for i in range(ncycles):
    if cycles % 10**3 == 0:
        of10 = int(cycles*10/ncycles)
        progress = "="*of10
        spaces = " "*(10-of10)
        padding = " "*8
        sys.stdout.write(f"\rOn iteration: {cycles:^10,}[{progress:<10}]")
        sys.stdout.flush()
    for n, node in enumerate(cur_nodes):
        zzzs = set(mm.p2node_zzzs[node])
        if n == 0:
            zs_in_all = zzzs
        else:
            zs_in_all = zs_in_all.intersection(zzzs)
    if len(zs_in_all) > 0:
        print(f"Success! All current nodes end in Z on the {min(zs_in_all)} iteration of cycle {cycles}")
        tot_steps = cycles*len(mm.directions) + min(zs_in_all)
        print(f"This is a total of: {tot_steps:,}")
        print(f"For references here's the zs in all index(es): {zs_in_all}")
        break
    else:
        #Repeat!
        cur_nodes = [mm.p2node_maps[node] for node in cur_nodes]
        cycles += 1

end = datetime.datetime.now()
elapsed = end - start
print(f"Process took: {elapsed}")

On iteration: 1,999,999,000[========= ]Process took: 4 days, 0:26:18.627134


In [89]:
cur_node

NameError: name 'cur_node' is not defined

In [ ]:
# cur_nodes = starting_nodes
# #Initialize zs in all to zs from first node


# for n, node in enumerate(cur_nodes):
#     zzzs = set(mm.p2node_zzzs[cur_nodes[0]])
#     if n == 0:
#         zs_in_all = zzzs
#     else:
#         zs_in_all = zs_in_all.intersection(zzzs)
# if len(zs_in_all) > 0:
#     print(f"Success! All current nodes end in Z on the {min(zs_in_all)} iteration of cycle {cycle}")
# else:
#     #Repeat!

## V3 - Detect Cycles, Allowing for Complete Modular Arithmetic

In [61]:
# ex = starting_nodes[0]
# starting_point = ex
# zzzs = mm.p2node_zzzs[ex]
# print(ex)
# print(len(zzzs), zzzs[:5])
# print(len(mm.directions))
# print(f"End node: {mm.p2node_maps[ex]}")


def modularize_node_path(starting_node):
    cur_node = starting_node
    iterations = 1

    node_order = []
    end_nodes = set()
    while (cur_node not in end_nodes):
        next_node = mm.p2node_maps[cur_node]
        iterations += 1
        end_nodes.add(cur_node)
        node_order.append(cur_node)
        cur_node = next_node
        if iterations > 10**8:
            print("Terminated early.")
            break
        elif iterations % 100 == 0:
            sys.stdout.write(f"\rOn iteration: {iterations:,}     ")
            sys.stdout.flush()
    print(f"Directions cycle after {iterations:,} iterations ending with: {next_node}")
    cyclical_after_n_steps = iterations*len(mm.directions)
    print(f"This is a total of {cyclical_after_n_steps:,} since directions are {len(mm.directions):,} long.")
    
    cycle_index_start = node_order.index(cur_node)
    node_order.append(cur_node)#append to verify the loop; will have to remove later.
    print(cycle_index_start)

    print(len(node_order), "\n",
          "First 5 direction starting nodes: ", node_order[:5], "\n"
          "Last 5 direction starting nodes: ", node_order[-5:])
    
    dir_iters_in_cycle = len(node_order[:-1]) - cycle_index_start
    print(f"Starting at {starting_node}, the path will cycle every {dir_iters_in_cycle*len(mm.directions)} steps starting after {cyclical_after_n_steps} steps")
    #additional feature engineer
    loop_length = len(node_order[cycle_index_start:])*len(mm.directions)
    loop_offset = len(node_order[:cycle_index_start])*len(mm.directions)
    zebras = []
    for iters, node in enumerate(node_order[cycle_index_start:-1]):
        zs = mm.p2node_zzzs[node]
        if zs != []:
            zs = [zi + iters*len(mm.directions) for zi in zs]
            zebras += zs #append to list
    params = {"node_order" : node_order,
              "cycle_index_start" : cycle_index_start,
              "loop_length":loop_length,
             "loop_offset":loop_offset,
             "zebras":zebras}
    return params


node_mod_params = {}
for node in starting_nodes:
    params = modularize_node_path(node)
    node_mod_params[node] = params
    print("\n")

Directions cycle after 61 iterations ending with: QHQ
This is a total of 18,727 since directions are 307 long.
1
61 
 First 5 direction starting nodes:  ['STA', 'QHQ', 'NXS', 'LLT', 'DVJ'] 
Last 5 direction starting nodes:  ['FDL', 'SHK', 'VCK', 'XGZ', 'QHQ']
Starting at STA, the path will cycle every 18113 steps starting after 18727 steps


Directions cycle after 69 iterations ending with: CRR
This is a total of 21,183 since directions are 307 long.
1
69 
 First 5 direction starting nodes:  ['AAA', 'CRR', 'CVB', 'KSH', 'SMD'] 
Last 5 direction starting nodes:  ['CLH', 'JHM', 'TSN', 'ZZZ', 'CRR']
Starting at AAA, the path will cycle every 20569 steps starting after 21183 steps


Directions cycle after 73 iterations ending with: GVP
This is a total of 22,411 since directions are 307 long.
1
73 
 First 5 direction starting nodes:  ['GPA', 'GVP', 'KBT', 'SJB', 'STP'] 
Last 5 direction starting nodes:  ['VFT', 'QKC', 'DHQ', 'LLZ', 'GVP']
Starting at GPA, the path will cycle every 21797 ste

In [65]:
for node in starting_nodes:
    print(f"Starting node: {node}")
    params = node_mod_params[node]
    node_order = params['node_order']
    loop_offset = params['loop_offset']
    loop_length = params['loop_length']


    #Print outputs
    print(len(node_order), "\n",
              "First 5 direction starting nodes: ", node_order[:5], "\n"
              "Last 5 direction starting nodes: ", node_order[-5:])


    print(f"Second loop starts after: {len(node_order)*len(mm.directions):<10,}")
    print(f"Or loop offset: {loop_offset:<10,}")
    print(f"Loop length: {loop_length:<10,}")
    print(f"Zebras in cycle: {params['zebras']}")
    # for temp_node in node_order[loop_start_idx:-1]:


Starting node: STA
61 
 First 5 direction starting nodes:  ['STA', 'QHQ', 'NXS', 'LLT', 'DVJ'] 
Last 5 direction starting nodes:  ['FDL', 'SHK', 'VCK', 'XGZ', 'QHQ']
Second loop starts after: 18,727    
Or loop offset: 307       
Loop length: 18,420    
Zebras in cycle: [17806]
Starting node: AAA
69 
 First 5 direction starting nodes:  ['AAA', 'CRR', 'CVB', 'KSH', 'SMD'] 
Last 5 direction starting nodes:  ['CLH', 'JHM', 'TSN', 'ZZZ', 'CRR']
Second loop starts after: 21,183    
Or loop offset: 307       
Loop length: 20,876    
Zebras in cycle: [20262]
Starting node: GPA
73 
 First 5 direction starting nodes:  ['GPA', 'GVP', 'KBT', 'SJB', 'STP'] 
Last 5 direction starting nodes:  ['VFT', 'QKC', 'DHQ', 'LLZ', 'GVP']
Second loop starts after: 22,411    
Or loop offset: 307       
Loop length: 22,104    
Zebras in cycle: [21490]
Starting node: LKA
45 
 First 5 direction starting nodes:  ['LKA', 'XFX', 'GRM', 'RQQ', 'JDT'] 
Last 5 direction starting nodes:  ['GML', 'VBR', 'GTM', 'HTZ', 'XFX

In [66]:
[1,2,3,4,5].index(4)

3

In [77]:
for node in starting_nodes:
    params = node_mod_params[node]
    print(f"Starting node: {node}")
    print(f"Loop length: {params['loop_length']:<10,}")
    print(f"Zebras in cycle: {len(params['zebras'])}")
    print(f"where: {params['zebras'][0] % params['loop_length']} mod {params['loop_length']}")
    print(f"Diff: {params['loop_length'] - params['zebras'][0]}")
    print("\n")

Starting node: STA
Loop length: 18,420    
Zebras in cycle: 1
where: 17806 mod 18420
Diff: 614


Starting node: AAA
Loop length: 20,876    
Zebras in cycle: 1
where: 20262 mod 20876
Diff: 614


Starting node: GPA
Loop length: 22,104    
Zebras in cycle: 1
where: 21490 mod 22104
Diff: 614


Starting node: LKA
Loop length: 13,508    
Zebras in cycle: 1
where: 12894 mod 13508
Diff: 614


Starting node: DFA
Loop length: 24,560    
Zebras in cycle: 1
where: 23946 mod 24560
Diff: 614


Starting node: KKA
Loop length: 22,718    
Zebras in cycle: 1
where: 22104 mod 22718
Diff: 614




Thus, multiply all of the cycle lenghts together, subtract the joint offset (307) and the share mod diff (614)

In [85]:
import numpy as np

In [90]:
prod = 1
for node in starting_nodes:
    params = node_mod_params[node]
    prod = np.lcm(params['loop_length'], prod)
print(f"Product: {prod:,}")
print(f"Final answer: {prod+307-614:,}")

Product: 1,529,375,760
Final answer: 1,529,375,453


### Ex:

3 mod 4 
and 2 mod 3.....

both are 1 away from their modulator, so multiply together, they should be 1 away from joint product

answer: 11!

5 mod 6 would also work in this scenario, demonstrting need to find LCM
